Primeira coisa do pre-processamento é transformar os dados em um formato que o modelo de machine learning possa trabalhar. Nesse caso, e variaveis numericas.


IMPORTANDO AS BIBLIOTECAS

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


LENDO O ARQUIVO

In [9]:
df = pd.read_csv("C:/Users/55819/Downloads/Crime_prediction/data/raw/dataset_ocorrencias_delegacia_5.csv.")
df.head()

,id_ocorrencia,data_ocorrencia,bairro,tipo_crime,descricao_modus_operandi,arma_utilizada,quantidade_vitimas,quantidade_suspeitos,sexo_suspeito,idade_suspeito,orgao_responsavel,status_investigacao,latitude,longitude
0,OCR100000,2024-08-02 19:48:16.958668,Imbiribeira,Sequestro,Golpe Telefônico,Faca,2,0,Não Informado,53,Delegacia Casa Forte,Concluído,-8.111355,-34.943524
1,OCR100001,2022-10-29 19:48:16.958694,Boa Viagem,Homicídio,Fraude Online,Objeto Contundente,4,1,Não Informado,29,Delegacia Casa Forte,Arquivado,-8.082012,-34.861911
2,OCR100002,2023-03-12 19:48:16.958698,Santo Amaro,Furto,Invasão Residencial,Nenhum,1,4,Feminino,65,Delegacia Casa Forte,Arquivado,-8.096916,-34.898378
3,OCR100003,2025-04-18 19:48:16.958700,Afogados,Roubo,Arrombamento,Nenhum,1,4,Não Informado,30,Delegacia Torre,Concluído,-8.135066,-34.913460
4,OCR100004,2025-06-30 19:48:16.958702,Tamarineira,Roubo,Estupro Coletivo,Arma de Fogo,3,3,Masculino,20,Delegacia Casa Forte,Concluído,-8.074124,-34.933504


APAGAR COLUNAS QUE NAO SAO RELEVANTES PARA O MODELO

In [10]:
df = df.drop(['id_ocorrencia', 'latitude', 'longitude'], axis=1)
df.head()

,data_ocorrencia,bairro,tipo_crime,descricao_modus_operandi,arma_utilizada,quantidade_vitimas,quantidade_suspeitos,sexo_suspeito,idade_suspeito,orgao_responsavel,status_investigacao
0,2024-08-02 19:48:16.958668,Imbiribeira,Sequestro,Golpe Telefônico,Faca,2,0,Não Informado,53,Delegacia Casa Forte,Concluído
1,2022-10-29 19:48:16.958694,Boa Viagem,Homicídio,Fraude Online,Objeto Contundente,4,1,Não Informado,29,Delegacia Casa Forte,Arquivado
2,2023-03-12 19:48:16.958698,Santo Amaro,Furto,Invasão Residencial,Nenhum,1,4,Feminino,65,Delegacia Casa Forte,Arquivado
3,2025-04-18 19:48:16.958700,Afogados,Roubo,Arrombamento,Nenhum,1,4,Não Informado,30,Delegacia Torre,Concluído
4,2025-06-30 19:48:16.958702,Tamarineira,Roubo,Estupro Coletivo,Arma de Fogo,3,3,Masculino,20,Delegacia Casa Forte,Concluído


Feature Engineering (Criação de Novas Variáveis) adicionando a variável ano e mes que pode ser útil para prever crimes com base no tempo.


In [11]:
# Convertendo a coluna 'data_ocorrencia' para datetime
df['data_ocorrencia'] = pd.to_datetime(df['data_ocorrencia'])

# Extraindo ano e mês da data
df['ano'] = df['data_ocorrencia'].dt.year
df['mes'] = df['data_ocorrencia'].dt.month

# Excluindo a coluna 'data_ocorrencia' original (não será necessária no modelo)
df.drop(columns=['data_ocorrencia'], inplace=True)

COPIANDO O DATAFRAME PRINCIPAL PARA USAR A COPIA EM OUTROS MODELOS.

In [12]:
df_copy = df.copy()


In [13]:
# Agrupar os dados por bairro, mês e ano e contar o número de ocorrências

df_crimes = df.groupby(['bairro', 'ano', 'mes']).size().reset_index(name='quantidade_crimes')

# Verificando a transformação
df_crimes.head()

,bairro,ano,mes,quantidade_crimes
0,Afogados,2022,8,2
1,Afogados,2022,9,16
2,Afogados,2022,10,16
3,Afogados,2022,11,20
4,Afogados,2022,12,13


Escalonamento das Variáveis

In [14]:
from sklearn.preprocessing import StandardScaler

# Escalonando as variáveis numéricas
scaler = StandardScaler()
df[['quantidade_vitimas', 'quantidade_suspeitos', 'ano', 'mes']] = scaler.fit_transform(
    df[['quantidade_vitimas', 'quantidade_suspeitos', 'ano', 'mes']]
)

SEPARANDO EM TREINO E TESTE

In [15]:
from sklearn.model_selection import train_test_split

# Variável dependente (target) e variáveis independentes (features)
X = df_crimes.drop(columns=["quantidade_crimes"])  # Usando as outras variáveis como features
y = df_crimes["quantidade_crimes"]  # Prevendo a quantidade de crimes

# Dividindo em treinamento (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificando as dimensões dos dados
print(f"Tamanho do treino: {X_train.shape}, Tamanho do teste: {X_test.shape}")


Tamanho do treino: (293, 3), Tamanho do teste: (74, 3)


In [16]:
print(X_train.columns.tolist())


['bairro', 'ano', 'mes']


OPTAMOS PELO MODELO RANDOMFORESTREGRESSOR - treinando so com as features que eu considero importantes que sao bairros, ano e mes.

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib


# Features de acordo com seu X_train atual
categorical_features = ["bairro"]
numeric_features = ["ano", "mes"]

# Pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", StandardScaler(), numeric_features),
    ]
)

# Modelo
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(random_state=42))
])

# Treinamento
model.fit(X_train, y_train)

# Previsão
y_pred = model.predict(X_test)

# Avaliação
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.2f}")

joblib.dump(model, "modelo_random_forest_reduzido.pkl")



MAE: 3.93
R²: 0.02


['modelo_random_forest_reduzido.pkl']

OPTAMOS PELO MODELO RANDOMFORESTREGRESSOR - treinando com as features que sao relevantes 

In [ ]:

df_agg = df.groupby([
    "bairro",
    "tipo_crime",
    "arma_utilizada",
    "quantidade_vitimas",
    "quantidade_suspeitos",
    "sexo_suspeito",
    "idade_suspeito",
    "ano",
    "mes"
]).size().reset_index(name="quantidade_crimes")


feature_columns = [
    "bairro",
    "tipo_crime",
    "arma_utilizada",
    "quantidade_vitimas",
    "quantidade_suspeitos",
    "sexo_suspeito",
    "idade_suspeito",
    "ano",
    "mes"
]

target_column = "quantidade_crimes"

X = df_agg[feature_columns]
y = df_agg[target_column]

 
#  Separar treino e teste

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

 
#  Atualizar colunas categóricas e numéricas

categorical_features = [
    "bairro",
    "tipo_crime",
    "arma_utilizada",
    "sexo_suspeito"
]

numeric_features = ["quantidade_vitimas", "quantidade_suspeitos", "idade_suspeito", "ano", "mes"]

 
#  Pré-processamento

cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")
num_imputer = SimpleImputer(strategy="mean")

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", Pipeline([
            ("imputer", cat_imputer),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), categorical_features),
        ("num", Pipeline([
            ("imputer", num_imputer),
            ("scaler", StandardScaler())
        ]), numeric_features)
    ]
)

 
#  Pipeline Random Forest

model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1))
])

#  Treinar o modelo
model.fit(X_train, y_train)

 
#  Prever e avaliar

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Avaliação do modelo Random Forest:")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.2f}")

joblib.dump(model, "modelo_random_forest_all_features.pkl")



🌲 Avaliação do modelo Random Forest (colunas removidas):
MAE: 0.00
R²: 1.00


['modelo_random_forest_all_features.pkl']

INSISTS DESSAS DUAS MANEITAS DE TREINAR O MODELO 

1 - QUANDO EU SO USEI TRES COLUNAS PARA O MEU TREINO A MAE DEU MUITO ALTA OQUE É RUIM PARA O MEU MODELO PORQUE ELE ESTA ERRANDO 3,93 CRIMES POR BARRIO/MES. E ELA E O R AO QUADRADO DEU 2% E ISSO É MUITO BAIXO OQUE SIGNIFICA QUE ELE NAO ESTA CONSEGUINDO MELHORAR O MEU MODELO.

2- O OUTRO MODELO QUE EU TREINEI CONSEGUIO TER METRICASA MELHORES. PORQUE EU COLOQUEI MAIS FEACTURES RELEVANTES PARA O MODELO. A MAE DEU 1.48 OU SEJA ELE ERRA 1.48 CRIMES POR BARRIO/MES. E O R DEU  84% E ISSO SIGNIFICA QUE ELE ESTA CONSEGUINDO MELHORAR O MEU MODELO.

TESTANDO OUTRO MODELO QUE VAI SER O XGBOOST 

In [1]:
pip install xgboost

                                              0.0/56.8 MB ? eta -:--:--
                                              0.1/56.8 MB 2.6 MB/s eta 0:00:22
                                              0.1/56.8 MB 1.4 MB/s eta 0:00:40
                                              0.2/56.8 MB 2.0 MB/s eta 0:00:29
                                              0.5/56.8 MB 2.6 MB/s eta 0:00:23
                                              0.5/56.8 MB 2.4 MB/s eta 0:00:24
                                              1.0/56.8 MB 4.2 MB/s eta 0:00:14
     -                                        1.8/56.8 MB 5.7 MB/s eta 0:00:10
     -                                        2.2/56.8 MB 5.9 MB/s eta 0:00:10
     --                                       3.3/56.8 MB 8.0 MB/s eta 0:00:07
     --                                       4.2/56.8 MB 8.9 MB/s eta 0:00:06
     ---                                      5.6/56.8 MB 10.7 MB/s eta 0:00:05
     ----                                     6.4/56.8 MB 


[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, r2_score

 
# Preparar dados agregados (target)



df_agg = df_copy.groupby(["bairro", "ano", "mes"]).agg({
    "tipo_crime": "first",
    "arma_utilizada": "first",
    "quantidade_vitimas": "sum",
    "quantidade_suspeitos": "sum",
    "sexo_suspeito": "first",
    "idade_suspeito": "mean"
}).reset_index()

# Criar target
df_agg["quantidade_crimes"] = df_copy.groupby(["bairro", "ano", "mes"]).size().values


#  Todas as features

features_all = [
    "bairro", "tipo_crime", "arma_utilizada",
    "quantidade_vitimas", "quantidade_suspeitos",
    "sexo_suspeito", "idade_suspeito",
    "ano", "mes"
]

categorical_all = ["bairro", "tipo_crime", "arma_utilizada", "sexo_suspeito"]
numeric_all = ["quantidade_vitimas", "quantidade_suspeitos", "idade_suspeito", "ano", "mes"]

X_all = df_agg[features_all]
y_all = df_agg["quantidade_crimes"]

X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(
    X_all, y_all, test_size=0.2, random_state=42
)

preprocessor_all = ColumnTransformer(
    transformers=[
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), categorical_all),
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
        ]), numeric_all)
    ]
)

model_all = Pipeline(steps=[
    ("preprocessor", preprocessor_all),
    ("regressor", XGBRegressor(n_estimators=200, random_state=42, n_jobs=-1))
])

# Treinar
model_all.fit(X_train_all, y_train_all)

# Prever e avaliar
y_pred_all = model_all.predict(X_test_all)
mae_all = mean_absolute_error(y_test_all, y_pred_all)
r2_all = r2_score(y_test_all, y_pred_all)

print("XGBoost Todas as Features")
print(f"MAE: {mae_all:.2f}, R²: {r2_all:.2f}")

NameError: name 'df_copy' is not defined